# Introducción
A continuación se presenta un algoritmo para resolución de multiplicación de matrices. La implementación se realizó en MPI.

El flujo de trabajo consiste en un proceso master y N procesos esclavos. El proceso master se encarga de dividir en porciones las matrices que deben procesarse. Los esclavos, en base a estos límites definidos por el master, calculan su parte y le envían de nuevo al master los resultados.

El master recibe todas las porciones y muestra el resultado junto con las matrices iniciales.

# Armado del ambiente
No es necesario ningún setup dentro del cuaderno.

# Desarrollo

In [24]:
code = """
#include <mpi.h>
#include <stdio.h>
#include <string.h>

#define CANT_FILAS_A 12
#define CANT_COLUMNAS_A 12
#define CANT_FILAS_B 12
#define CANT_COLUMNAS_B 12

#define MASTER_TO_SLAVE_TAG 1
#define SLAVE_TO_MASTER_TAG 4

void inicializarMatrices();
void imprimirResultados();

int myRank; // id de proceso
int numProcs; // cantidad de procesos
int i, j, k;

double mat_a[CANT_FILAS_A][CANT_COLUMNAS_A];
double mat_b[CANT_FILAS_B][CANT_COLUMNAS_B];
double mat_result[CANT_FILAS_A][CANT_COLUMNAS_B];

double tiempoInicio;
double tiempoFin;

int limite_inf; // primer numero de fila de [A] que le corresponden al esclavo
int limite_sup; // ultimo numero de fila de [A] que le corresponden al esclavo
int porcion; // cantidad de filas de [A] que le corresponden a un esclavo

MPI_Status status; // store status of a MPI_Recv
MPI_Request request; //capture request of a MPI_Isend

int main(int argc, char** argv) {
  MPI_Init(NULL, NULL);

  MPI_Comm_rank(MPI_COMM_WORLD, &myRank);
  MPI_Comm_size(MPI_COMM_WORLD, &numProcs);

  inicializarMatrices();
  
  // master
  if (myRank == 0) {
    tiempoInicio = MPI_Wtime();

    // por cada proceso que no sea el master
    for (i = 1; i < numProcs; i++) { 
      // calculo el límite inferior y superior que le corresponde calcular
      porcion = (CANT_FILAS_A / (numProcs - 1)); 
      limite_inf = (i - 1) * porcion;

      // si estoy en el último proceso y no puedo dividir las columnas de A por la cantidad de esclavos
      if (((i + 1) == numProcs) && ((CANT_FILAS_A % (numProcs - 1)) != 0)) { 
        // el último proceso se lleva las filas que queden
        limite_sup = CANT_FILAS_A;
      } else {
        limite_sup = limite_inf + porcion; 
      }

      // envio el limite inferior al esclavo correspondiente
      MPI_Isend(&limite_inf, 1, MPI_INT, i, MASTER_TO_SLAVE_TAG, MPI_COMM_WORLD, &request);

      // envio el limite superior al esclavo correspondiente
      MPI_Isend(&limite_sup, 1, MPI_INT, i, MASTER_TO_SLAVE_TAG + 1, MPI_COMM_WORLD, &request);
    }
  }

  // esclavos calculan y envían los resultados
  if (myRank > 0) {
    // recibo límite inferior y superior del master
    MPI_Recv(&limite_inf, 1, MPI_INT, 0, MASTER_TO_SLAVE_TAG, MPI_COMM_WORLD, &status);
    MPI_Recv(&limite_sup, 1, MPI_INT, 0, MASTER_TO_SLAVE_TAG + 1, MPI_COMM_WORLD, &status);

    // calculo
    for (i = limite_inf; i < limite_sup; i++) {
      for (j = 0; j < CANT_COLUMNAS_B; j++) {
        for (k = 0; k < CANT_FILAS_B; k++) {
          mat_result[i][j] += (mat_a[i][k] * mat_b[k][j]);
        }
      }
    }

    // envio de nuevo el limite inferior y superior al master
    MPI_Isend(&limite_inf, 1, MPI_INT, 0, SLAVE_TO_MASTER_TAG, MPI_COMM_WORLD, &request);
    MPI_Isend(&limite_sup, 1, MPI_INT, 0, SLAVE_TO_MASTER_TAG + 1, MPI_COMM_WORLD, &request);

    // envio los resultados calculados de la porción que me correspondía calcular
    MPI_Isend(&mat_result[limite_inf][0], (limite_sup - limite_inf) * CANT_COLUMNAS_B, MPI_DOUBLE, 0, SLAVE_TO_MASTER_TAG + 2, MPI_COMM_WORLD, &request);
  }

  // master recolecta los resultados
  if (myRank == 0) {
    for (i = 1; i < numProcs; i++) {
      // recibe límite inferior y superior del esclavo
      MPI_Recv(&limite_inf, 1, MPI_INT, i, SLAVE_TO_MASTER_TAG, MPI_COMM_WORLD, &status);
      MPI_Recv(&limite_sup, 1, MPI_INT, i, SLAVE_TO_MASTER_TAG + 1, MPI_COMM_WORLD, &status);

      // recibo resultados del esclavo
      MPI_Recv(&mat_result[limite_inf][0], (limite_sup - limite_inf) * CANT_COLUMNAS_B, MPI_DOUBLE, i, SLAVE_TO_MASTER_TAG + 2, MPI_COMM_WORLD, &status);
    }
    tiempoFin = MPI_Wtime();
    printf("\\nTiempo de ejecución: %f [ms]\\n\\n", (tiempoFin - tiempoInicio) * 1000);
    imprimirResultados();
  }

  MPI_Finalize();
  return 0;
}

void inicializarMatrices()
{
    for (i = 0; i < CANT_FILAS_A; i++) {
        for (j = 0; j < CANT_COLUMNAS_A; j++) {
            mat_a[i][j] = i + j + 2;
        }
    }
    for (i = 0; i < CANT_FILAS_B; i++) {
        for (j = 0; j < CANT_COLUMNAS_B; j++) {
            mat_b[i][j] = i * j * 2;
        }
    }
}

void imprimirResultados()
{
    for (i = 0; i < CANT_FILAS_A; i++) {
        printf("\\n");
        for (j = 0; j < CANT_COLUMNAS_A; j++)
            printf("%8.2f  ", mat_a[i][j]);
    }
    printf("\\n\\n\\n");
    for (i = 0; i < CANT_FILAS_B; i++) {
        printf("\\n");
        for (j = 0; j < CANT_COLUMNAS_B; j++)
            printf("%8.2f  ", mat_b[i][j]);
    }
    printf("\\n\\n\\n");
    for (i = 0; i < CANT_FILAS_A; i++) {
        printf("\\n");
        for (j = 0; j < CANT_COLUMNAS_B; j++)
            printf("%8.2f  ", mat_result[i][j]);
    }
    printf("\\n\\n");
}
"""

text_file = open("hello.c", "w")
text_file.write(code)
text_file.close()

In [25]:
!mpicc -o hello hello.c

In [26]:
!mpirun --allow-run-as-root -n 12 hello


Tiempo de ejecución: 3.692061 [ms]


    2.00      3.00      4.00      5.00      6.00      7.00      8.00      9.00     10.00     11.00     12.00     13.00  
    3.00      4.00      5.00      6.00      7.00      8.00      9.00     10.00     11.00     12.00     13.00     14.00  
    4.00      5.00      6.00      7.00      8.00      9.00     10.00     11.00     12.00     13.00     14.00     15.00  
    5.00      6.00      7.00      8.00      9.00     10.00     11.00     12.00     13.00     14.00     15.00     16.00  
    6.00      7.00      8.00      9.00     10.00     11.00     12.00     13.00     14.00     15.00     16.00     17.00  
    7.00      8.00      9.00     10.00     11.00     12.00     13.00     14.00     15.00     16.00     17.00     18.00  
    8.00      9.00     10.00     11.00     12.00     13.00     14.00     15.00     16.00     17.00     18.00     19.00  
    9.00     10.00     11.00     12.00     13.00     14.00     15.00     16.00     17.00     18.00     19.00     20

# Tabla de pasos

Proceso master

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  include               | Importación de librerías.
CPU      |  MPI_Init()            | Inicializa MPI.
CPU      |  MPI_Comm_...()        | Obtiene rank y cantidad de procesos esclavos.
CPU      |  inicializarMatrices() | Inicializa las matrices originales
CPU      |  MPI_Wtime()           | Obtiene tiempo de inicio
CPU      |  for                   | Envía los límites inf y sup a cada esclavo
CPU      |  for                   | Recibe los resultados de cada esclavo
CPU      |  MPI_Wtime()           | Obtiene tiempo de fin
CPU      |  printf()              | Muestra el tiempo de procesamiento
CPU      |  imprimirResultados()  | Muestra las matrices originales y el resultado

Proceso esclavo

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  include               | Importación de librerías.
CPU      |  MPI_Init()            | Inicializa MPI.
CPU      |  MPI_Comm_...()        | Obtiene rank y cantidad de procesos esclavos.
CPU      |  inicializarMatrices() | Inicializa las matrices originales
CPU      |  MPI_Recv()            | Recibe el límite inf y sup que tiene que procesar
CPU      |  for, for              | Recorro y proceso la porción que me corresponde calcular
CPU      |  MPI_Send()            | Envío los resultados al master


# Conclusiones
Luego de ejecutar el programa jugando con la cantidad de procesos esclavos trabajando, podemos darnos cuenta que el algoritmo termina performando mejor cuando menos esclavos hay involucrados.

Esto puede atribuirse a que la arquitectura de trabajo consiste en procesos diferentes trabajando en paralelo. Por esto, los cambios de contexto cuestan más de lo que en un principio cuesta la tarea a resolver.

Un análisis más profundo podría hacerse intentando encontrar el punto de equilibrio entre cantidad de procesos trabajando y cantidad de trabajo a procesar. Con esto, se podría comprender si vale la pena el paralelizar el procesamiento en este tipo de algoritmos.

# Bibliografía

Message Passing with MPI

http://www.red-ricap.org/documents/1071192/1486573/MPI_Tutorial_03.pdf/fff6399a-1fd9-4326-8990-050986c44b8a


Multiplicación de matrices

https://www.problemasyecuaciones.com/matrices/multiplicar-matrices-producto-matricial-ejemplos-explicados-propiedades-matriz.html